In [1]:
import pandas as pd
import glob
from igc_lib.igc_lib import Flight
from pdflight import flight_to_dataframe,thermals_to_dataframe
flights = [Flight.create_from_file(i) for i in glob.glob("igc_files/*.igc")]
[flight.glider_type for flight in flights]
#flight = Flight.create_from_file("/home/muenchm/thinclient_drives/C:/Users/muenchm/Desktop/2025-05-01 Wolfgang Janowitsch 562460.igc")


TypeError: '<=' not supported between instances of 'float' and 'NoneType'

In [ ]:
fl = pd.concat(flight_to_dataframe(flight) for flight in flights)
fl

In [ ]:
df = pd.concat(thermals_to_dataframe(flight)[1:] for flight in flights)
th = df.loc[fl.loc[df.index, "task"]]
th.index.get_level_values("datetime")

In [ ]:
def shift_datetime(i, dt):
    return ((i[0] + ),) + i[1:]

In [ ]:
df = pd.DataFrame()
df["t"] = th.index.get_level_values("datetime") + th
for i in th.index:
    df.loc[i, "v"] = (fl.loc[shift_datetime(i, th.loc[i])].alt - fl.loc[i].alt) / th.loc[i].total_seconds()
    df.loc[i, "n"] = (fl.loc[shift_datetime(i, pd.Timedelta(seconds=40))].alt - fl.loc[i].alt) / 40
    df.loc[i, "x"] = (fl.loc[shift_datetime(i, th.loc[i] - pd.Timedelta(seconds=60))].alt - fl.loc[shift_datetime(i, th.loc[i] - pd.Timedelta(seconds=20))].alt) / -40
df["dn"] = df.n / df.v
df["dx"] = df.x / df.v
df[df.v > 1]

In [ ]:
print(df[df.v > 1].groupby(["code", "id"]).dn.mean())

In [ ]:
print(df[df.v > 1].groupby(["code", "id"]).dx.mean())

In [ ]:
print(df.v.mean())